In [1]:
import os
from glob import glob
import numpy as np 
from models.VAE import VariationalAutoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
section = 'vae'
run_id='0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.makedirs(os.path.join(RUN_FOLDER, 'viz'))
    os.makedirs(os.path.join(RUN_FOLDER, 'images'))
    os.makedirs(os.path.join(RUN_FOLDER, 'weights'))

mode = 'build'

DATA_FOLDER = 'o:/Dataset/Large-scale-CelebFaces-Dataset/Img/'
#filenames = np.array(glob(os.path.join(DATA_FOLDER, '*.jpg')))

In [24]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32
filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))
NUM_IMAGES = len(filenames)

In [25]:
print(NUM_IMAGES)
print(DATA_FOLDER)

202599
o:/Dataset/Large-scale-CelebFaces-Dataset/Img/


In [26]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(
    DATA_FOLDER
    ,target_size = INPUT_DIM[:2]
    ,batch_size = BATCH_SIZE
    ,shuffle =True
    ,class_mode = 'input'
    ,subset='training'
    )

Found 202599 images belonging to 1 classes.


In [27]:
vae = VariationalAutoencoder(
    input_dim=INPUT_DIM
    , encoder_conv_filters=[32,64,64,64]
    , encoder_conv_kernel_size=[3,3,3,3]
    , encoder_conv_strides=[2,2,2,2]
    , decoder_conv_t_filters=[64,64,32,3]
    , decoder_conv_t_kernel_size=[3,3,3,3]
    , decoder_conv_t_strides=[2,2,2,2]
    , z_dim=200
    , use_batch_norm=True
    , use_dropout=True
)

if mode=='build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [28]:
vae.encoder.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_7[0][0]      
____________________________________________________________________________________________

In [29]:
vae.decoder.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 200)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_11 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 16, 16, 64)        0   

In [30]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR =10000
EPOCHS = 50
PRINT_EVERY_N_BATCHES = 1000
INITIAL_EPOCHS=0

In [31]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [32]:
vae.train_with_generator(
    data_flow
    ,epochs= EPOCHS
    ,steps_per_epoch=NUM_IMAGES/BATCH_SIZE
    ,run_folder=RUN_FOLDER
    ,print_every_n_batches=PRINT_EVERY_N_BATCHES
    ,initial_epoch=INITIAL_EPOCHS
)

W0609 12:08:37.664214 17332 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.
Epoch 1/50
Exception in thread Thread-74:
Traceback (most recent call last):
  File "c:\users\rvill\appdata\local\programs\python\python37\Lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\rvill\appdata\local\programs\python\python37\Lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "O:\virtuelenv\generative\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 843, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "O:\virtuelenv\generative\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 822, in pool_fn
    initargs=(seqs, None, get_worker_id_queue()))
  File "c:\users\rvill\appdata\local\programs\python\python37\Lib\multiprocessing\context.py", line 119, in 